In [ ]:
!pip install -q neo4j

In [ ]:
from neo4j import GraphDatabase
import pandas as pd

In [ ]:
# Read the CSV file into a pandas DataFrame
file_path = "recipe_data.csv"  # Replace with the path to your CSV file
data = pd.read_csv(file_path)

# Preview the data
data.head()

,Recipe_ID,Recipe_Name,Cuisine_Type,Ingredients_List,Preparation_Steps,Cooking_Time_Minutes,Difficulty_Level,User_ID,User_Preferences,User_Rating,User_Feedback,Date_Prepared,Season,Occasion,Serving_Size,Calories_Per_Serving,Allergen_Information,Cost_Per_Serving,Popularity_Score,Seasonal_Availability
0,R001,Beef Tacos,Mexican,"Beef, Taco Shells, Lettuce, Cheese, Tomato",Cook beef; Prepare toppings; Assemble tacos.,55,Medium,U539,NaN,3,Feedback 1,2024-07-06,Spring,Party,4,509,NaN,14.12,402,No
1,R002,Spaghetti Bolognese,Italian,"Spaghetti, Ground Beef, Tomato Sauce, Onion, G...",Boil pasta; Cook beef; Add sauce; Combine.,39,Easy,U538,NaN,2,Feedback 2,2024-02-24,Winter,Casual,4,275,Gluten,16.07,183,No
2,R003,Chicken Curry,Indian,"Chicken, Curry Powder, Coconut Milk, Onion, Ga...",Cook chicken; Add spices; Pour coconut milk; S...,36,Medium,U324,Gluten-Free,4,Feedback 3,2024-02-14,Winter,Party,5,293,NaN,11.97,137,Yes
3,R004,Chicken Curry,Indian,"Chicken, Curry Powder, Coconut Milk, Onion, Ga...",Cook chicken; Add spices; Pour coconut milk; S...,45,Medium,U548,Gluten-Free,2,Feedback 4,2024-04-13,Spring,Casual,6,580,NaN,10.36,246,Yes
4,R005,Vegetable Stir Fry,Chinese,"Broccoli, Carrot, Bell Pepper, Soy Sauce, Garlic",Chop veggies; Stir-fry; Add sauce; Serve.,85,Easy,U990,Vegan,5,Feedback 5,2024-08-03,Fall,Dinner,4,641,Soy,12.68,386,No


In [ ]:
# Connect to the Neo4j database
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def query(self, query, parameters=None):
        with self._driver.session() as session:
            result = session.run(query, parameters)
            return result.data()

In [ ]:
# Replace these with your Neo4j credentials
uri = ""  # Change to your Neo4j URI
user = "neo4j"                 # Your Neo4j username
password = ""          # Your Neo4j password

conn = Neo4jConnection(uri, user, password)

In [ ]:
def load_data_to_neo4j(data):
    # Create constraints (existing ones)
    conn.query("CREATE CONSTRAINT IF NOT EXISTS FOR (r:Recipe) REQUIRE r.id IS UNIQUE;")
    conn.query("CREATE CONSTRAINT IF NOT EXISTS FOR (u:User) REQUIRE u.id IS UNIQUE;")
    conn.query("CREATE CONSTRAINT IF NOT EXISTS FOR (i:Ingredient) REQUIRE i.name IS UNIQUE;")
    conn.query("CREATE CONSTRAINT IF NOT EXISTS FOR (a:Allergen) REQUIRE a.name IS UNIQUE;")
    conn.query("CREATE CONSTRAINT IF NOT EXISTS FOR (o:Occasion) REQUIRE o.name IS UNIQUE;")
    conn.query("CREATE CONSTRAINT IF NOT EXISTS FOR (s:Season) REQUIRE s.name IS UNIQUE;")
    # Add new constraint for Cuisine
    conn.query("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Cuisine) REQUIRE c.name IS UNIQUE;")

    for _, row in data.iterrows():
        # Create Cuisine node first
        cuisine_query = """
        MERGE (c:Cuisine {name: $cuisine})
        """
        conn.query(cuisine_query, {
            "cuisine": row["Cuisine_Type"]
        })

        # Modified Recipe node creation to remove cuisine as property
        recipe_query = """
        MERGE (r:Recipe {
            id: $id,
            name: $name,
            difficulty: $difficulty,
            cooking_time: $cooking_time,
            serving_size: $serving_size,
            calories: $calories,
            cost: $cost,
            popularity_score: $popularity_score,
            seasonal_availability: $seasonal_availability
        })
        WITH r
        MATCH (c:Cuisine {name: $cuisine})
        MERGE (r)-[:BELONGS_TO]->(c)
        """
        conn.query(recipe_query, {
            "id": row["Recipe_ID"],
            "name": row["Recipe_Name"],
            "cuisine": row["Cuisine_Type"],
            "difficulty": row["Difficulty_Level"],
            "cooking_time": row["Cooking_Time_Minutes"],
            "serving_size": row["Serving_Size"],
            "calories": row["Calories_Per_Serving"],
            "cost": row["Cost_Per_Serving"],
            "popularity_score": row["Popularity_Score"],
            "seasonal_availability": row["Seasonal_Availability"]
        })

        # Rest of the existing code remains the same...
        user_query = """
        MERGE (u:User {id: $id})
        """
        conn.query(user_query, {"id": row["User_ID"]})

        user_recipe_query = """
        MATCH (u:User {id: $user_id}), (r:Recipe {id: $recipe_id})
        MERGE (u)-[rel:RATED {rating: $rating}]->(r)
        """
        conn.query(user_recipe_query, {
            "user_id": row["User_ID"],
            "recipe_id": row["Recipe_ID"],
            "rating": row["User_Rating"]
        })

        if pd.notna(row["User_Preferences"]) and row["User_Preferences"] != "None":
            preferences_query = """
            MATCH (u:User {id: $user_id}), (r:Recipe {id: $recipe_id})
            MERGE (u)-[pref:PREFERS {preference: $preference}]->(r)
            """
            conn.query(preferences_query, {
                "user_id": row["User_ID"],
                "recipe_id": row["Recipe_ID"],
                "preference": row["User_Preferences"]
            })

        ingredients = [i.strip() for i in row["Ingredients_List"].split(",")]
        for ingredient in ingredients:
            ingredient_query = """
            MERGE (i:Ingredient {name: $name})
            WITH i
            MATCH (r:Recipe {id: $recipe_id})
            MERGE (i)-[:USED_IN]->(r)
            """
            conn.query(ingredient_query, {
                "name": ingredient,
                "recipe_id": row["Recipe_ID"]
            })

        if not pd.isna(row["Allergen_Information"]):
            allergens = [a.strip() for a in row["Allergen_Information"].split(",") if a.strip()]
            for allergen in allergens:
                allergen_query = """
                MERGE (a:Allergen {name: $name})
                WITH a
                MATCH (r:Recipe {id: $recipe_id})
                MERGE (a)-[:PRESENT_IN]->(r)
                """
                conn.query(allergen_query, {
                    "name": allergen,
                    "recipe_id": row["Recipe_ID"]
                })

        if not pd.isna(row["Occasion"]):
            occasion_query = """
            MERGE (o:Occasion {name: $name})
            WITH o
            MATCH (r:Recipe {id: $recipe_id})
            MERGE (r)-[:SUITABLE_FOR]->(o)
            """
            conn.query(occasion_query, {
                "name": row["Occasion"],
                "recipe_id": row["Recipe_ID"]
            })

        if not pd.isna(row["Season"]):
            season_query = """
            MERGE (s:Season {name: $name})
            WITH s
            MATCH (r:Recipe {id: $recipe_id})
            MERGE (r)-[:BEST_IN]->(s)
            """
            conn.query(season_query, {
                "name": row["Season"],
                "recipe_id": row["Recipe_ID"]
            })

        if not pd.isna(row["Preparation_Steps"]):
            steps = [step.strip() for step in row["Preparation_Steps"].split(".") if step.strip()]
            for i, step in enumerate(steps, 1):
                step_query = """
                MERGE (p:PreparationStep {step_number: $step_number, description: $description})
                WITH p
                MATCH (r:Recipe {id: $recipe_id})
                MERGE (r)-[:HAS_STEP]->(p)
                """
                conn.query(step_query, {
                    "step_number": i,
                    "description": step,
                    "recipe_id": row["Recipe_ID"]
                })

In [ ]:
# Call the function to load the data into Neo4j
load_data_to_neo4j(data)